### Import data

### Student Information
Name:杜義榤

Student ID:113232506

GitHub ID:EJDU21

Kaggle name:yijieeeee

Kaggle private scoreboard snapshot:
![2024-12-05 140046.png](2024-12-05 140046.png)


1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [ ]:
### Begin Assignment Here
from IPython.display import Image
Image('./img/pic0.png')#印出競賽排名截圖

In [4]:
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [5]:
data = []
with open('./dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
 
f.close()


In [6]:
emotion = pd.read_csv('./dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('./dm-2024-isa-5810-lab-2-homework/data_identification.csv')

In [7]:
df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})
df = df.merge(data_identification, on='tweet_id', how='left')

train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']

In [8]:
train_data = train_data.merge(emotion, on='tweet_id', how='left')
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)
y_train_data = train_data['emotion']
X_train_data = train_data.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)

In [9]:
print(len(train_data))
print(len(X_train_data))

1449182
1449182


In [10]:
test_data = test_data.merge(emotion, on='tweet_id', how='left')
test_data.drop_duplicates(subset=['text'], keep=False, inplace=True)
y_test_data = test_data['emotion']
X_test_data = test_data.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)

print(len(test_data))
print(len(X_test_data))

411972
411972


In [ ]:
import sentencepiece as spm
import nltk
nltk.download('wordnet')
import random
from nltk.corpus import wordnet
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd


# **1. 同義詞替換**
def synonym_replacement(sentence, n=2):
    words = sentence.split()
    if len(words) < n:  # 如果句子詞數量少於 n，調整 n
        n = len(words)
    new_sentence = words[:]
    random_indices = random.sample(range(len(words)), n)
    for idx in random_indices:
        synonyms = wordnet.synsets(words[idx])
        if synonyms:
            synonym = random.choice(synonyms).lemmas()[0].name()
            new_sentence[idx] = synonym
    return ' '.join(new_sentence)

# **2. 隨機刪除**
def random_deletion(sentence, p=0.1):
    words = sentence.split()
    if len(words) == 1:
        return sentence  # 保證至少有一個詞
    new_sentence = [word for word in words if random.random() > p]
    return ' '.join(new_sentence) if new_sentence else random.choice(words)

# **3. 回譯**
def back_translate(sentence, src_lang="en", tgt_lang="fr"):
    tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}")
    model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}")
    translated = tokenizer.encode(sentence, return_tensors="pt", max_length=512, truncation=True)
    translated = model.generate(translated, max_length=512, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    # 回譯
    tokenizer = MarianTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{tgt_lang}-{src_lang}")
    model = MarianMTModel.from_pretrained(f"Helsinki-NLP/opus-mt-{tgt_lang}-{src_lang}")
    translated = tokenizer.encode(translated_text, return_tensors="pt", max_length=512, truncation=True)
    translated = model.generate(translated, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# **4. 資料增強**
def augment_data(data, synonym_ratio=0.3, delete_ratio=0.2, backtranslate_ratio=0.3):
    augmented_data = []

    # 計算每個增強方法需要處理的數據量
    synonym_count = int(len(data) * synonym_ratio)
    delete_count = int(len(data) * delete_ratio)
    backtranslate_count = int(len(data) * backtranslate_ratio)
    original_count = len(data) - (synonym_count + delete_count + backtranslate_count)

    # 隨機分配數據
    data = data.sample(frac=1, random_state=42)  # 打亂數據
    synonym_data = data.iloc[:synonym_count]
    delete_data = data.iloc[synonym_count:synonym_count + delete_count]
    backtranslate_data = data.iloc[synonym_count + delete_count:synonym_count + delete_count + backtranslate_count]
    original_data = data.iloc[synonym_count + delete_count + backtranslate_count:]

    # 應用同義詞替換
    for text in synonym_data['text']:
        augmented_data.append(synonym_replacement(text, n=2))

    # 應用隨機刪除
    for text in delete_data['text']:
        augmented_data.append(random_deletion(text, p=0.1))

    # 應用回譯
    for text in backtranslate_data['text']:
        augmented_data.append(back_translate(text, src_lang="en", tgt_lang="fr"))

    # 保留原始數據
    for text in original_data['text']:
        augmented_data.append(text)

    return pd.DataFrame({'text': augmented_data})

# **5. 應用增強到訓練數據**
augmented_X_train = augment_data(X_train_data, synonym_ratio=0.3, delete_ratio=0.2, backtranslate_ratio=0.3)

print(augmented_X_train.head())





ImportError: DLL load failed while importing _sentencepiece: 找不到指定的程序。

Language Embedding

In [ ]:
import torch
import os
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

####################################################################
# Load Model & Tokenizer
model_name = "dunzhang/stella_en_1.5B_v5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
print("Model and tokenizer loaded successfully!")
####################################################################
# move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.cuda.empty_cache()
####################################################################
#讓dataloader讀取資料
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts.tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]
####################################################################
# 分批處理
def encode_texts_in_batches(texts, tokenizer, model, device, batch_size=32, max_length=512):
    num_samples = len(texts)
    all_embeddings = []

    for i in tqdm(range(0, num_samples, batch_size), desc="Encoding batches"):
        batch_texts = texts[i:i+batch_size].tolist()
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
            all_embeddings.append(embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

# word embedding
X_train_embeddings = encode_texts_in_batches(X_train_data['text'], tokenizer, model, device, batch_size=32)
X_test_embeddings = encode_texts_in_batches(X_test_data['text'], tokenizer, model, device, batch_size=32)
############################################################################################################
#save tokenizer
save_path = "./saved_tokenizer"
os.makedirs(save_path, exist_ok=True) 
tokenizer.save_pretrained(save_path)
print(f"Tokenizer saved to {save_path}")
###################################################################
#save embedding
np.save('X_train_embeddings.npy', X_train_embeddings)
np.save('X_test_embeddings.npy', X_test_embeddings)


True
1
NVIDIA GeForce RTX 4070 Ti SUPER


c:\Users\iverson\miniconda3\envs\kaggle_as2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and tokenizer loaded successfully!


In [ ]:
from sklearn.manifold import TSNE

# t-SNE 降维
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_train_tsne = tsne.fit_transform(X_train_embeddings)
# plot
plt.figure(figsize=(10, 8))
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y, cmap='Spectral', s=5)
plt.colorbar()
plt.title("t-SNE Visualization of Word Embeddings")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show()

In [ ]:
#load embedding
X_train_embeddings = np.load('X_train_embeddings.npy', mmap_mode='r')
X_test_embeddings = np.load('X_test_embeddings.npy', mmap_mode='r')
# one hot
le = LabelEncoder()
y_train = le.fit_transform(y_train_data)

1536
8


In [ ]:
#train textCnn( pytorch )
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm

# hyper parameter
embedding_dim = X_train_embeddings.shape[1]  # word embedding 维度
num_classes = len(set(y_train_data))
batch_size = 64
num_epochs = 50
learning_rate = 0.001
#讓dataloader讀取資料
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
# encode label
y_train = le.fit_transform(y_train_data)

# create train dataset
train_dataset = EmbeddingDataset(X_train_embeddings, y_train)

# dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#train
class TextCNN(nn.Module):
    def __init__(self, embedding_dim, num_classes, kernel_sizes=[3, 4, 5], num_channels=256):
        super(TextCNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_channels, (1, embedding_dim))  # model 
            for _ in kernel_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(len(kernel_sizes) * num_channels, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1).unsqueeze(2)  # 增加两个维度，形狀從 [batch_size, embedding_dim] -> [batch_size, 1, 1, embedding_dim]
        x = [torch.relu(conv(x)).squeeze(3).squeeze(2) for conv in self.convs]  # (batch_size, num_channels)
        x = torch.cat(x, 1)  
        x = self.dropout(x)
        logits = self.fc(x)
        return logits


# 初始化模型
model = TextCNN(embedding_dim, num_classes)
print(model)
model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # train(w/o 進度條)
# for epoch in range(num_epochs):
#     model.train()
#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(torch.device('cuda')), targets.to(torch.device('cuda'))
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    # 初始化訓練進度條
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for inputs, targets in pbar:
            # 將數據移到 GPU
            inputs, targets = inputs.to(torch.device('cuda')), targets.to(torch.device('cuda'))
            
            # forward
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # backward & optimizer
            loss.backward()
            optimizer.step()
            
            # loss
            epoch_loss += loss.item()
            
            # 更新進度條
            pbar.set_postfix(loss=loss.item())
    
    # 打印當前 epoch 的平均loss
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(train_loader)}")


TextCNN(
  (convs): ModuleList(
    (0-2): 3 x Conv2d(1, 256, kernel_size=(1, 1536), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=768, out_features=8, bias=True)
)


Epoch 1/50: 100%|██████████| 22644/22644 [02:02<00:00, 184.12batch/s, loss=1.32] 


Epoch 1/50, Average Loss: 1.232000875854172


Epoch 2/50: 100%|██████████| 22644/22644 [02:02<00:00, 185.08batch/s, loss=1.04] 


Epoch 2/50, Average Loss: 1.1887551269526406


...

Epoch 48/50, Average Loss: 1.1066247623067003


Epoch 49/50: 100%|██████████| 22644/22644 [02:04<00:00, 181.64batch/s, loss=0.923]


Epoch 49/50, Average Loss: 1.1055362051113653


Epoch 50/50: 100%|██████████| 22644/22644 [02:04<00:00, 182.30batch/s, loss=1.21] 

Epoch 50/50, Average Loss: 1.1055227710262159


In [ ]:
#textCnn predict
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
#讓dataloader讀取資料
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32)
# test embedding
test_dataset = EmbeddingDataset(X_test_embeddings)

#dataloader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 切换到评估模式
model.eval()

# 存储预测结果
all_predictions = []

# 推理
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)  # 获取预测的类别索引
        all_predictions.extend(predictions.cpu().numpy())

# 反编碼預測结果
y_pred_labels = le.inverse_transform(all_predictions)

# 建立輸出 DataFrame
submission = pd.DataFrame({
    'id': test_data['tweet_id'],  # 假设 test_data 包含 tweet_id 信息
    'emotion': y_pred_labels
})

# 保存到 CSV 文件
submission.to_csv('./save/submission_TextCNN.csv', index=False)
print("Predictions saved to './save/submission_TextCNN.csv'")


Predictions saved to './save/submission_TextCNN.csv'


In [ ]:
#train transformer ( pytorch )
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# hyper parameter
embedding_dim = X_train_embeddings.shape[1]  # 词嵌入维度
num_classes = len(set(y_train_data))  # 类别数量
batch_size = 64
num_epochs = 20
learning_rate = 0.001
#讓dataloader讀取資料
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
# encode label 
y_train = le.fit_transform(y_train_data)  

# create training dataset
train_dataset = EmbeddingDataset(X_train_embeddings, y_train)

# create dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

class TransformerClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes, num_heads=8, num_layers=2, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.embedding_dim = embedding_dim

        # Transformer Encoder
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,  # input 維度
            nhead=num_heads,  
            dim_feedforward=embedding_dim * 4, 
            dropout=dropout,
            activation='relu'
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer,
            num_layers=num_layers  # Transformer Encoder 層數
        )

        # Classification head
        self.fc = nn.Linear(embedding_dim, num_classes)  

    def forward(self, x):
        # 輸入形狀: [batch_size, embedding_dim]
        x = x.unsqueeze(1)  # 添加一个维度 [batch_size, seq_len=1, embedding_dim]
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)# Transformer Encoder
        x = x.mean(dim=0)
        logits = self.fc(x)
        return logits

def get_scheduler(optimizer, d_model, warmup_steps=4000):
    def lr_lambda(step):
        if step == 0:
            return 1e-7
        return (d_model ** -0.5) * min(step ** -0.5, step * warmup_steps ** -1.5)
    return LambdaLR(optimizer, lr_lambda)

# 初始化 Transformer 模型
model = TransformerClassifier(embedding_dim=embedding_dim, num_classes=num_classes, num_heads=8, num_layers=2)
print(model)
model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 使用學習率调度器
d_model = embedding_dim  # 嵌入维度
warmup_steps = 4000
scheduler = get_scheduler(optimizer, d_model, warmup_steps)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    # 初始化进度条
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for inputs, targets in pbar:
            # 將數據移到 GPU
            inputs, targets = inputs.to(torch.device('cuda')), targets.to(torch.device('cuda'))
            
            # forward
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # backward & optimize
            loss.backward()
            optimizer.step()
            scheduler.step()
            # 梯度剪裁
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            # loss
            epoch_loss += loss.item()
            
            # 更新進度條
            pbar.set_postfix(loss=loss.item())

    scheduler.step()
    # 打印當前 epoch 的平均損失
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss / len(train_loader)}")



c:\Users\iverson\miniconda3\envs\kaggle_as2\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


TransformerClassifier(
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=1536, out_features=1536, bias=True)
    )
    (linear1): Linear(in_features=1536, out_features=6144, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=6144, out_features=1536, bias=True)
    (norm1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1536, out_features=1536, bias=True)
        )
        (linear1): Linear(in_features=1536, out_features=6144, bias=True)
        (dropout): Dropout(p=0.1, i

Epoch 1/20:   3%|▎         | 761/22644 [00:09<04:47, 76.15batch/s, loss=1.9] 


KeyboardInterrupt: 

In [ ]:
#transformer predict (pytorch)
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
#讓dataloader讀取資料
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx], dtype=torch.float32)

test_dataset = EmbeddingDataset(X_test_embeddings)#test embedding 
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)#dataloader

# 切换模型到評估模式
model.eval()

# 儲存預測结果
all_predictions = []

# 推理
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)
        all_predictions.extend(predictions.cpu().numpy())

# 反編碼預測結果
y_pred_labels = le.inverse_transform(all_predictions)

# 建立 DataFrame
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels
})

# 保存到 CSV 文件
submission.to_csv('./save/submission_TF.csv', index=False)
print("Predictions saved to './save/submission_TextCNN.csv'")

Predictions saved to './save/submission_TextCNN.csv'


In [ ]:
#train LogisticRegression (sklearn)
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(max_iter=1000, random_state=42)
clf2.fit(X_train_embeddings, y_train)
################################################################################################################
#predict LogisticRegression (sklearn)
y_pred_LogisticRegression = clf2.predict(X_test_embeddings)
y_pred_labels = le.inverse_transform(y_pred_LogisticRegression)
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels})
#################################################################################################################
# 保存到 CSV 文件
submission.to_csv('./save/submission_LogisticRegression.csv', index=False)

In [ ]:
#train LGBMClassifier (sklearn)
import lightgbm as lgb
clf4 = lgb.LGBMClassifier(random_state=42, verbose=-1)
clf4.fit(X_train_embeddings, y_train)
y_pred_LGBMClassifier = clf4.predict(X_test_embeddings)
################################################################################################################
#predict LGBMClassifier (sklearn)
y_pred_labels = le.inverse_transform(y_pred_LGBMClassifier)
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels})
##################################################################################################################
# 保存到 CSV 文件
submission.to_csv('./save/submission_LGBMClassifier.csv', index=False)


In [ ]:
#train MLPClassifier (sklearn)
from sklearn.neural_network import MLPClassifier
clf6 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42, warm_start=True)
clf6.fit(X_train_embeddings, y_train)
y_pred_MLPClassifier = clf6.predict(X_test_embeddings)
################################################################################################################
#predict MLPClassifier (sklearn)
y_pred_labels = le.inverse_transform(y_pred_MLPClassifier)
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels})
#################################################################################################################
# 保存到 CSV 文件
submission.to_csv('./save/submission_MLPClassifier.csv', index=False)